In [1]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, dijkstra, format_path

from pprint import pprint
import numpy as np


importing Jupyter notebook from dijkstra.ipynb
('1', '2', '3', '4', '5n', '5s', '6', '7', '8', '9', '10', '10b', '11n', '11s', '12', '13', '14', '15', '16', '17', '19', '20', '20b', '21', '22', '23', '24', '25/1', '25/2', '26', '31', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22')
[[inf inf  7. ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [ 7. inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf  4. inf]
 [inf inf inf ...  4. inf inf]
 [inf inf inf ... inf inf inf]]
X1 -> X2 -> 9 -> 7
9.0
2: [2, 36, 35, 35, 33, 33, 32, 47, 32, 7, 37, 42, 43, 44, 45, 41, 40, 40, 17, 46, 50, 52, 52, -1, 49, 48, 47, 51, 27, 50, 25, 32, 9, 9, 33, 34, 34, 9, 37, 38, 39, 16, 39, 38, 45, 46, 47, 48, 49, 50, 51, 21, 23]
1: [55.0, 55.0, 48.0, 47.0, 45.0, 44.0, 40.0, 35.0, 40.0, 37.0, 42.0, 48.0, 45.0, 39.0, 38.0, 51.0, 47.0, 47.0, 50.0, 35.0, 22.0, 8.0, 3.0, 0, 26.0, 31.0, 32.0, 20.0, 22.0, 22.0, 33.0, 44.0, 39.0

In [2]:
def init_tamponi_per_reparto():
    tamponi_per_reparto = np.array([0, 0, 0, 3, 0, 0, 0, 6, 0, 0, 0, 0, 12, 0, 0, 0, 7, 0, 0, 0, 10, 0,
        0, 0, 3, 0, 30, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]).astype(int)

    return tamponi_per_reparto

In [3]:
tamponi_per_reparto = init_tamponi_per_reparto()
print(tamponi_per_reparto)

[ 0  0  0  3  0  0  0  6  0  0  0  0 12  0  0  0  7  0  0  0 10  0  0  0
  3  0 30  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]


In [4]:
def estrai_reparti_con_tamponi(tamponi_per_reparto):
    return [i for i,t in enumerate(tamponi_per_reparto) if t > 0]

In [5]:
reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)

print(reparti_con_tamponi)

for index,node in enumerate(reparti_con_tamponi):
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha {tamponi_per_reparto[node]} tamponi")

[3, 7, 12, 16, 20, 24, 26, 28]
0. Reparto con indice 3 e nome '4' ha 3 tamponi
1. Reparto con indice 7 e nome '7' ha 6 tamponi
2. Reparto con indice 12 e nome '11n' ha 12 tamponi
3. Reparto con indice 16 e nome '14' ha 7 tamponi
4. Reparto con indice 20 e nome '19' ha 10 tamponi
5. Reparto con indice 24 e nome '22' ha 3 tamponi
6. Reparto con indice 26 e nome '24' ha 30 tamponi
7. Reparto con indice 28 e nome '25/2' ha 4 tamponi


In [6]:
## SCORE MATRIX

# conteggio dei tamponi in ogni nodo dato un path (oppure un nodo di partenza) 
def compute_tamponi_in_nodo(p, prev, tamponi_per_reparto, tamponi_per_path, start_node=None):
    if prev is None and start_node is not None:
        _, prev = np.array(dijkstra(arc_matrix, start_node))
        tamponi_per_path = -1 * np.ones(len(prev))
    
    path = []
    path.append(p)

    prev = np.array(prev).astype(int)
    while p != -1:
        p = prev[p]
        path.append(p)

    path = path[::-1] # reverse
    path = path[1:] # remove start node

    temp_tamponi = 0
    for p in path:
        if tamponi_per_path[p] == -1:
            # print(p,"aggiungo questo reparto al dizionario")
            temp_tamponi += tamponi_per_reparto[p]
            tamponi_per_path[p] = temp_tamponi
        else:
            temp_tamponi = tamponi_per_path[p]
        
        # print(p, nodes[p], tamponi_per_reparto[p], temp_tamponi)
    
    return tamponi_per_path
        

def compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto, tamponi_per_reparto, criterio="density"):
    time, prev = np.array(dijkstra(arc_matrix, indice_reparto))
    time = time[reparti_con_tamponi]
    
    if criterio == "density":
        ## TAMPONI PER PATH
        tamponi_per_path = -1 * np.ones(len(prev))
        for reparto in reparti_con_tamponi:
            tamponi_per_path = compute_tamponi_in_nodo(reparto, prev, tamponi_per_reparto, tamponi_per_path)
        ## TAMPONI IN NODO FINALE
        # tamponi_per_path = tamponi_per_reparto.copy()
        tamponi_per_path = tamponi_per_path[reparti_con_tamponi]
        
        # print(tamponi_per_path)
        density_score = np.divide(tamponi_per_path, time, out=-np.inf*np.ones_like(time), where=time!=0)
        return density_score
    if criterio == "time":
        return time
    raise ValueError(f"{criterio} non è un criterio valido. Usa 'density' o 'time'")

# tamponi / minuto
def compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    return compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)

def compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    reparti_con_tamponi = np.array(reparti_con_tamponi)
    score_matrix = np.zeros((len(reparti_con_tamponi), len(reparti_con_tamponi)))
    for i, indice_reparto in enumerate(reparti_con_tamponi):
        score_matrix[i] = compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto, tamponi_per_reparto)
    return score_matrix

In [7]:
def retrieve_best_score(score_matrix, index=None):
    if index is not None: # per score_matrix
        return np.argmax(score_matrix[index]), np.max(score_matrix[index])
    else:
        return np.argmax(score_matrix), np.max(score_matrix)

In [8]:
score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)
for index,node in enumerate(reparti_con_tamponi):
    best_index, best_score = retrieve_best_score(score_matrix, index)
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha best score in reparto {best_index}. indice {reparti_con_tamponi[best_index]} con nome '{nodes[reparti_con_tamponi[best_index]]}' con score {best_score}")

0. Reparto con indice 3 e nome '4' ha best score in reparto 6. indice 26 con nome '24' con score 2.2941176470588234
1. Reparto con indice 7 e nome '7' ha best score in reparto 6. indice 26 con nome '24' con score 7.2
2. Reparto con indice 12 e nome '11n' ha best score in reparto 6. indice 26 con nome '24' con score 3.2
3. Reparto con indice 16 e nome '14' ha best score in reparto 6. indice 26 con nome '24' con score 2.5294117647058822
4. Reparto con indice 20 e nome '19' ha best score in reparto 6. indice 26 con nome '24' con score 2.5
5. Reparto con indice 24 e nome '22' ha best score in reparto 6. indice 26 con nome '24' con score 3.3
6. Reparto con indice 26 e nome '24' ha best score in reparto 1. indice 7 con nome '7' con score 7.2
7. Reparto con indice 28 e nome '25/2' ha best score in reparto 6. indice 26 con nome '24' con score 1.4166666666666667


In [9]:
def check_reparto_intermedio(start_node, next_index, reparti_con_tamponi, tamponi_per_reparto):
    # controllo: se all'interno del path c'è un nodo con tamponi, allora scelgo quello
    
    path, _ = get_shortest_path(arc_matrix, start_node, reparti_con_tamponi[next_index])
    
    nodo_finale = None
    for p in path[1:-1]:
        if p in reparti_con_tamponi and tamponi_per_reparto[p] > 0:
            nodo_finale = next_index
            next_index = reparti_con_tamponi.index(p)
            break
    
    return next_index, nodo_finale

def get_next_node(start_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto):
    index = reparti_con_tamponi.index(start_node)
    next_index, score = retrieve_best_score(score_matrix, index)
    
    next_index, nodo_finale = check_reparto_intermedio(start_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
    
    score_matrix[:,index] = -np.inf
    return reparti_con_tamponi[next_index], score, nodo_finale

In [10]:
DIMENSIONE_PIASTRA = 96 # tamponi
massimo_ritardo = 0 # minuti
def stop_condition(residual_time, tamponi_trasportati, step, reparti_con_tamponi):
    return residual_time <= -massimo_ritardo or tamponi_trasportati >= DIMENSIONE_PIASTRA or step >= len(reparti_con_tamponi)

In [11]:
def build_tsp_path(arc_matrix, tamponi_per_reparto_orig, residual_time, tamponi_trasportati, current_node, lab):
    reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto_orig)
    tamponi_per_reparto = tamponi_per_reparto_orig.copy()
    path = []
    
    step = 0
    # se il nodo di partenza non è un nodo con tamponi, allora trovo il nodo con tamponi con score più alto
    if current_node not in reparti_con_tamponi:
        
        score_row = compute_score_row(arc_matrix, reparti_con_tamponi, current_node, tamponi_per_reparto)
        
        ## TODO: controlla se accorpabile con get_next_node
        next_index, score = retrieve_best_score(score_row)
        next_index, nodo_finale = check_reparto_intermedio(current_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
        next_node = reparti_con_tamponi[next_index]
        ## END TODO
        
        distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        
        ## DEBUG:
        # print(f"{step}. '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")
        ##
        
        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        current_node = next_node
    
    step += 1
    lab_time_row = compute_score_row(arc_matrix, reparti_con_tamponi, lab, tamponi_per_reparto, criterio="time")
    lab_return_time = lab_time_row[reparti_con_tamponi.index(current_node)]
    while not stop_condition(residual_time - lab_return_time, tamponi_trasportati, step, reparti_con_tamponi):
        score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)
        next_node, score, nodo_finale = get_next_node(current_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto)
        
        distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        
        ## DEBUG:
        # print(f"{step}. '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")
        ##
        
        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        # print(f"Tempo di ritorno al laboratorio da current_node: {lab_time_row[reparti_con_tamponi.index(current_node)]}")
        # print(f"Tempo di ritorno al laboratorio da next_node: {lab_time_row[reparti_con_tamponi.index(next_node)]}")
        current_node = next_node
        step += 1
        lab_time_row = compute_score_row(arc_matrix, reparti_con_tamponi, lab, tamponi_per_reparto, criterio="time")
        lab_return_time = lab_time_row[reparti_con_tamponi.index(current_node)]
            
    # path = path[:-1] # remove last node (probabilmente)
    
    path += [lab]
        
    
    return path

In [12]:
start_node = nodes.index('4')
residual_time = 120
tamponi_trasportati = 0
lab = nodes.index('9')

path = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, start_node, lab)
print(f"Path: {format_path(path)}")

Path: 7 -> 24 -> 11n -> 14 -> 19 -> 22 -> 25/2 -> 9


In [13]:
# TODO
# provare a fare spawnare tamponi prima che termini il piano iniziale
# SCORE deve tenere conto di: (tamponi / tempo) * (?)
# + time_row tra i reparti con tamponi e laboratorio
# + aggiungere tempo di percorrenza (corridoio) A -> B (es 3 su 10 minuti percorsi)
#    + se devo andare in C e il percorso è A -> B -> C, allora per arrivare in C ci metto 3 minuti in meno
#    + se il percorso non prevede di passare per B, allora ci metto 3 minuti in più

start_node = nodes.index('X2')
residual_time = 120
tamponi_trasportati = 0
lab = nodes.index('7')

path = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, start_node, lab)
print(f"Path: {format_path(path)}")

0. 'X2' -> '7': 6.0 tamponi / 4.0 minuti = 4.0 score  (nodo intermedio verso 24). Tempo residuo: 116.0, Tamponi trasportati: 6.0.
Path: 7 -> 24 -> 11n -> 14 -> 19 -> 22 -> 25/2 -> 9


In [14]:
lab_score_row = compute_score_row(arc_matrix, reparti_con_tamponi, lab, tamponi_per_reparto, criterio='time')

print(lab_score_row)
for index,node in enumerate(reparti_con_tamponi):
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha tempo di percorrenza verso il laboratorio di {lab_score_row[index]} minuti")

[10.  2.  8. 10. 21. 15.  7. 29.]
0. Reparto con indice 3 e nome '4' ha tempo di percorrenza verso il laboratorio di 10.0 minuti
1. Reparto con indice 7 e nome '7' ha tempo di percorrenza verso il laboratorio di 2.0 minuti
2. Reparto con indice 12 e nome '11n' ha tempo di percorrenza verso il laboratorio di 8.0 minuti
3. Reparto con indice 16 e nome '14' ha tempo di percorrenza verso il laboratorio di 10.0 minuti
4. Reparto con indice 20 e nome '19' ha tempo di percorrenza verso il laboratorio di 21.0 minuti
5. Reparto con indice 24 e nome '22' ha tempo di percorrenza verso il laboratorio di 15.0 minuti
6. Reparto con indice 26 e nome '24' ha tempo di percorrenza verso il laboratorio di 7.0 minuti
7. Reparto con indice 28 e nome '25/2' ha tempo di percorrenza verso il laboratorio di 29.0 minuti


In [16]:
tamponi_per_reparto = [0]*len(nodes)
reparto = nodes.index('22')
tamponi_per_reparto[reparto] += 10
# reparto = nodes.index('2')
# print(reparti_con_tamponi.index(reparto))

residual_time = 119

tamponi_trasportati = 0

current_position = nodes.index('X1')

lab = nodes.index('9')

path = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, current_position, lab)

print(f"Path: {format_path(path)}")
# reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)
# lab_time_row = compute_score_row(arc_matrix, reparti_con_tamponi, lab, tamponi_per_reparto, criterio="time")
# print(lab_time_row)

0. 'X1' -> '22': 10.0 tamponi / 22.0 minuti = 0.45454545454545453 score . Tempo residuo: 97.0, Tamponi trasportati: 10.0.
Path: 22 -> 9
